In [ ]:
import torch
import tqdm
import pickle
import random
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
augments = []
base_items = []
full_items = []
units = []
reroll_tokens = ['CAN_REROLL', 'CANT_REROLL']
action_tokens = ['ACTION_BUY', 'ACTION_SELL', 'ACTION_MOVE', 'ACTION_ITEM',
                 'ACTION_AUGMENT', 'ACTION_REROLL', 'ACTION_NONE']
other_tokens = ['GO','EOS', 'MASK', 'PAD', 'HERO_BOUNDARY', 'BENCH_BOUNDARY', 'SHOP_BOUNDARY',
                'ITEM_BOUNDARY', 'AUGMENT_BOUNDARY', 'ITEM_SLOT', 'AUGMENT_SLOT', 'HERO_SLOT']

with open('augments.csv', 'r') as f:
    for line in f:
        # Strip newline and commas, then split by comma and join characters
        tokens = line.strip().split(',')
        string = ''.join(tokens)
        augments.append(string)
with open('base_items.csv', 'r') as f:
    for line in f:
        # Strip newline and commas, then split by comma and join characters
        tokens = line.strip().split(',')
        string = ''.join(tokens)
        base_items.append(string)
with open('full_items.csv', 'r') as f:
    for line in f:
        # Strip newline and commas, then split by comma and join characters
        tokens = line.strip().split(',')
        string = ''.join(tokens)
        full_items.append(string)
with open('units.csv', 'r') as f:
    for line in f:
        # Strip newline and commas, then split by comma and join characters
        tokens = line.strip().split(',')
        string = ''.join(tokens)
        units.append(string)
augments[augments.index('Cower"" Weaklings!')] = 'Cower, Weaklings!'
augments[augments.index('One"" Two"" Five!')] = 'One, Two, Five!'
augments[augments.index('10""000 IQ')] = '10,000 IQ'
augments[augments.index('One Buff"" Two Buff')] = 'One Buff, Two Buff'
tokens = augments + base_items + full_items + units + reroll_tokens + action_tokens + other_tokens
hero_boundary, bench_boundary, shop_boundary, item_boundary, augment_boundary, reroll, decision = 45, 56, 62, 78, 82, 83, 84

In [ ]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
with open('inv_vocab.pkl', 'rb') as f:
    inv_vocab = pickle.load(f)
vocab

{'Ziggs': 0,
 'Pair of Fours': 1,
 'Blazing Soul II': 2,
 "Zhonya's Paradox": 3,
 'Illaoi': 4,
 'Cower, Weaklings!': 5,
 "Slammin'+": 6,
 'ReinFOURcement': 7,
 'Adaptive Strikes': 8,
 'Rapidfire Crest': 9,
 'Syndicate Emblem': 10,
 "Nashor's Tooth": 11,
 'Statikk Shiv': 12,
 'ITEM_SLOT': 13,
 'Roll The Dice': 14,
 'Iron Assets': 15,
 'SHOP_BOUNDARY': 16,
 'Giant Slayer': 17,
 'Golden Fleece': 18,
 'Slayer Crown': 19,
 'One For All II': 20,
 'Recurve Bow': 21,
 'Senna': 22,
 "I'm the Carry Now": 23,
 'Prismatic Ticket': 24,
 'CAN_REROLL': 25,
 'Chain Vest': 26,
 'HERO_BOUNDARY': 27,
 'Missed Connections': 28,
 'Lucky Gloves': 29,
 'Bastion Crown': 30,
 'Marksman Circlet': 31,
 'Placebo': 32,
 'Bastion Crest': 33,
 'Cooking Pot': 34,
 'Marksman Crest': 35,
 'Eye For An Eye+': 36,
 'Void Swarm': 37,
 'Slayer Emblem': 38,
 'Jhin': 39,
 'New High Score': 40,
 'Starry Night+': 41,
 'Vi': 42,
 'Dynamo Circlet': 43,
 "Pandora's Items III": 44,
 'Anima Visage': 45,
 'Slayer Crest': 46,
 'Zephyr

In [ ]:
def weighted_random_choice(arr1, arr2, weight=0.9):
    """
    Selects a random index from arr1 or arr2 with higher probability for arr1.

    Args:
    - arr1: First array (higher probability selection).
    - arr2: Second array (lower probability selection).
    - weight: Probability of choosing arr1 (default 80%).

    Returns:
    - (index, selected_array) where index is from the chosen array.
    """
    total_length = len(arr1) + len(arr2)

    # Choose which array to pick from
    from_arr1 = random.choices([True, False], weights=[weight, 1 - weight])[0]

    if from_arr1:
        index = random.randint(0, len(arr1) - 1)
        return arr1[index]
    else:
        index = random.randint(0, len(arr2) - 1)
        return arr2[index]
def make_board():
  board = ['GO']
  board_heros = []
  bench_items = []
  bench_heros = []
  random_board_size = random.randint(4, 11)
  for i in range(11):
      if i < random_board_size:
          board.append(random.choice(units))
          board_heros.append(board[-1])
          random_items = weighted_random_choice([0,1],[2,3], weight=.95)
          for j in range(3):
              if j < random_items:
                  board.append(weighted_random_choice(full_items, base_items, weight=.95))
              else:
                  board.append('ITEM_SLOT')
      elif i == random_board_size:
          open_slot = weighted_random_choice(['PAD'], ['HERO_SLOT'], weight =1)
          if open_slot == 'HERO_SLOT':
              board.append(open_slot)
              board.append('ITEM_SLOT')
              board.append('ITEM_SLOT')
              board.append('ITEM_SLOT')
          else:
              board.append('PAD')
              board.append('PAD')
              board.append('PAD')
              board.append('PAD')
      else:
          board.append('PAD')
          board.append('PAD')
          board.append('PAD')
          board.append('PAD')
  board.append('HERO_BOUNDARY')
  random_bench_size = random.randint(0,10)
  for i in range(10):
      if i < random_bench_size:
          board.append(random.choice(units))
          bench_heros = board[-1]
      else:
          board.append('PAD')
  board.append('BENCH_BOUNDARY')
  for i in range(5):
      board.append(random.choice(units))
  board.append('SHOP_BOUNDARY')
  random_item_size = random.randint(5,15)
  for i in range(15):
      if i < random_item_size:
          board.append(weighted_random_choice(base_items,full_items, weight=.97))
          bench_items.append(board[-1])
      else:
          board.append('ITEM_SLOT')
  board.append('ITEM_BOUNDARY')
  random_augment_size = random.randint(0,3)
  for i in range(3):
      if i < random_augment_size:
          board.append(random.choice(augments))
      else:
          board.append('AUGMENT_SLOT')
  board.append('AUGMENT_BOUNDARY')
  board.append(random.choice(reroll_tokens))
  board.append('ACTION_ITEM')
  possible_items = [item for item in bench_items if item in base_items]
  first_item = random.choice(possible_items)
  board.append(first_item)
  possible_items.remove(first_item)
  board.append(random.choice(possible_items))
  board.append('MASK')
  board.append('PAD')
  board.append('EOS')
  return board

def make_penalty(board):
  penalty_vector = np.zeros(len(vocab))
  board_units = set([unit for idx,unit in enumerate(board[1:hero_boundary]) if unit in units and (board[idx+1] == 'ITEM_SLOT' or board[idx+2] == 'ITEM_SLOT' or board[idx+3] == 'ITEM_SLOT')])
  weight = 1/ len(board_units)
  for unit in board_units:
    penalty_vector[vocab[unit]] = weight
  return penalty_vector

In [ ]:
dataset_list = []
penalties_list = []

for _ in tqdm.tqdm(range(200704)):
    board = make_board()
    penalty = make_penalty(board)
    dataset_list.append(board)
    penalties_list.append(penalty)

# Convert to DataFrames after
dataset = pd.DataFrame(dataset_list, columns=range(90))
penalties = pd.DataFrame(penalties_list, columns=range(len(vocab)))

100%|██████████| 200704/200704 [00:22<00:00, 9117.43it/s]


In [ ]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,GO,Zed,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Seraphine,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Nidalee,...,Young and Wild and Free,Table Scraps,AUGMENT_BOUNDARY,CAN_REROLL,ACTION_ITEM,Negatron Cloak,Recurve Bow,MASK,PAD,EOS
1,GO,Naafiri,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Fiddlesticks,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Aurora,...,AUGMENT_SLOT,AUGMENT_SLOT,AUGMENT_BOUNDARY,CANT_REROLL,ACTION_ITEM,Recurve Bow,Giant's Belt,MASK,PAD,EOS
2,GO,Naafiri,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Vi,Horizon Focus,ITEM_SLOT,ITEM_SLOT,Urgot,...,AUGMENT_SLOT,AUGMENT_SLOT,AUGMENT_BOUNDARY,CAN_REROLL,ACTION_ITEM,Sparring Gloves,Negatron Cloak,MASK,PAD,EOS
3,GO,Graves,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Naafiri,Marksman Emblem,ITEM_SLOT,ITEM_SLOT,Jax,...,Pandora's Items III,AUGMENT_SLOT,AUGMENT_BOUNDARY,CAN_REROLL,ACTION_ITEM,Giant's Belt,Recurve Bow,MASK,PAD,EOS
4,GO,Vayne,Quicksilver,ITEM_SLOT,ITEM_SLOT,Kog'Maw,Jak'sho the Protean,ITEM_SLOT,ITEM_SLOT,Illaoi,...,Category Five,AUGMENT_SLOT,AUGMENT_BOUNDARY,CAN_REROLL,ACTION_ITEM,Recurve Bow,Tear of the Goddess,MASK,PAD,EOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200699,GO,Gragas,Thief's Gloves,ITEM_SLOT,ITEM_SLOT,Galio,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Galio,...,Branching Out+,AUGMENT_SLOT,AUGMENT_BOUNDARY,CANT_REROLL,ACTION_ITEM,Needlessly Large Rod,Sparring Gloves,MASK,PAD,EOS
200700,GO,Senna,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Shaco,ITEM_SLOT,ITEM_SLOT,ITEM_SLOT,Rengar,...,AUGMENT_SLOT,AUGMENT_SLOT,AUGMENT_BOUNDARY,CAN_REROLL,ACTION_ITEM,Giant's Belt,B.F. Sword,MASK,PAD,EOS
200701,GO,Vayne,Rosethorn Vest,ITEM_SLOT,ITEM_SLOT,Kindred,Warmog's Pride,ITEM_SLOT,ITEM_SLOT,Mordekaiser,...,AUGMENT_SLOT,AUGMENT_SLOT,AUGMENT_BOUNDARY,CAN_REROLL,ACTION_ITEM,B.F. Sword,Chain Vest,MASK,PAD,EOS
200702,GO,Draven,Fist of Fairness,ITEM_SLOT,ITEM_SLOT,Shaco,Aegis of the Legion,Spear of Hirana,ITEM_SLOT,Veigar,...,Memory Bank,AUGMENT_SLOT,AUGMENT_BOUNDARY,CAN_REROLL,ACTION_ITEM,Recurve Bow,Sparring Gloves,MASK,PAD,EOS


In [ ]:
penalties

,0,1,2,3,4,5,6,7,8,9,...,536,537,538,539,540,541,542,543,544,545
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
2,0.142857,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200699,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
200700,0.000000,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
200701,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0
200702,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0


In [ ]:
class MaskedTokenDataset(Dataset):
    def __init__(self, sequences, vocab, penalties):
        self.sequences = sequences
        self.vocab = vocab
        self.penalties = penalties

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        ids = self.sequences[idx]
        input_ids = [self.vocab[t] for t in ids]
        penalty = self.penalties[idx]
        return {'input':torch.tensor(input_ids), 'penalty': torch.tensor(penalty)}

def convert_test(sequence, vocab):
    input_ids = [vocab[t] for t in sequence]
    return torch.tensor(input_ids)

In [ ]:
class MaskedTokenTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=4, dim_feedforward=512, dropout=.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(90, d_model)
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Multiple heads, each for different outputs
        self.head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        positions = torch.arange(x.size(1), device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.pos_embedding(positions)
        x = self.dropout(x)

        x = self.transformer(x)

        # Apply different heads to the output for different tasks
        logits = self.head(x)

        return logits

In [ ]:
def custom_loss(outputs, target_indices, expected_classes):
    """
    outputs: Tensor of shape (batch_size, seq_len, vocab_size)
    target_indices: Tensor of shape (batch_size,) — the index of the token to supervise in each sequence
    expected_classes: Tensor of shape (batch_size,) — the expected class ID at that token
    """
    batch_size = outputs.size(0)

    # Get the logits at the target positions
    selected_logits = outputs[torch.arange(batch_size), target_indices]  # shape: (batch_size, vocab_size)
    # Apply cross-entropy loss at these positions
    loss = nn.functional.cross_entropy(selected_logits, expected_classes)
    return loss

In [ ]:
epochs =15
batch_size = 2048
sequences = dataset.values.tolist()
expected = penalties.values.tolist()

# Now use this in your dataset
data = MaskedTokenDataset(sequences, vocab, expected)
dataloader = DataLoader(data, batch_size=batch_size, drop_last=True)

model = MaskedTokenTransformer(len(vocab)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    tot_penalty = 0
    for inputs in dataloader:
        input_ids = inputs['input'].to(device)
        expected_tensor = inputs['penalty'].to(device)
        optimizer.zero_grad()
        logits = model(input_ids)
        loss= custom_loss(logits, decision+3, expected_tensor)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

Epoch 1/15 - Loss: 4.4026
Epoch 2/15 - Loss: 2.8904
Epoch 3/15 - Loss: 2.2262
Epoch 4/15 - Loss: 2.0985
Epoch 5/15 - Loss: 2.0315
Epoch 6/15 - Loss: 1.9961
Epoch 7/15 - Loss: 1.9715
Epoch 8/15 - Loss: 1.9568
Epoch 9/15 - Loss: 1.9460
Epoch 10/15 - Loss: 1.9374
Epoch 11/15 - Loss: 1.9305
Epoch 12/15 - Loss: 1.9247
Epoch 13/15 - Loss: 1.9198
Epoch 14/15 - Loss: 1.9157
Epoch 15/15 - Loss: 1.9116


In [ ]:
def evaluate(model, input_ids, masked_index):
    """
    input_ids: Tensor of shape (1, seq_len) — single input sequence with a [MASK] token
    masked_index: int — the position of the masked token in the sequence
    tokenizer: optional — to decode predicted token id to string
    """
    model.eval()
    with torch.no_grad():
        logits = model(input_ids)  # shape: (1, seq_len, vocab_size)
        masked_logits = logits[0, masked_index]  # shape: (vocab_size,)
        unit_logits = [masked_logits[vocab[unit]].item() for unit in board[1:hero_boundary] if unit in units]
        #print(f'MIN: {board[shop_boundary+1+item_logits.index(min(item_logits))]}, {min(item_logits)}')
        for idx, logit in enumerate(masked_logits):
          if logit > min(unit_logits) and inv_vocab[idx] not in board[1:hero_boundary]:
            print(f"Token ID {idx:>5} | Logit: {logit:.8f} | Token: {inv_vocab[idx]}")

        predicted_id = masked_logits.argmax(dim=-1).item()
        print(f"Predicted token ID at index {masked_index}: {inv_vocab[predicted_id]}")

    return predicted_id

In [ ]:
for i in range(5):
  board = make_board()
  print(board[1:hero_boundary])
  input_ids = [vocab[t] for t in board]
  id = evaluate(model, torch.tensor(input_ids).unsqueeze(0).to(device), decision+3)

['Sejuani', 'Jeweled Gauntlet', 'ITEM_SLOT', 'ITEM_SLOT', 'Miss Fortune', "Bulwark's Oath", 'ITEM_SLOT', 'ITEM_SLOT', 'Sejuani', 'Divinicorp Emblem', 'ITEM_SLOT', 'ITEM_SLOT', 'Senna', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Shyvana', 'Hullcrusher', 'ITEM_SLOT', 'ITEM_SLOT', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Predicted token ID at index 87: Sejuani
['Jinx', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Shaco', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'LeBlanc', 'Infinity Force', 'ITEM_SLOT', 'ITEM_SLOT', 'Gragas', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Yuumi', 'Hand Of Justice', 'ITEM_SLOT', 'ITEM_SLOT', 'Braum', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Gragas', "Tactician's Crown", 'ITEM_SLOT', 'ITEM_SLOT', 'Samira', 'Spear of Hirana', 'ITEM_SLOT', 'ITEM_SLOT', 'Shaco', 'Vanguard Emblem', 'ITEM_SLOT', 'ITEM_SLOT', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD',

In [ ]:
epochs=5
for epoch in range(epochs):
    total_loss = 0
    tot_penalty = 0
    for inputs in dataloader:
        input_ids = inputs['input'].to(device)
        expected_tensor = inputs['penalty'].to(device)
        optimizer.zero_grad()
        logits = model(input_ids)
        loss= custom_loss(logits, decision+3, expected_tensor)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

Epoch 1/5 - Loss: 1.8695
Epoch 2/5 - Loss: 1.8661
Epoch 3/5 - Loss: 1.8644
Epoch 4/5 - Loss: 1.8631
Epoch 5/5 - Loss: 1.8621


In [ ]:
for i in range(5):
  board = make_board()
  print(board[1:hero_boundary])
  input_ids = [vocab[t] for t in board]
  id = evaluate(model, torch.tensor(input_ids).unsqueeze(0).to(device), decision+3)

['Aurora', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Zac', 'Fishbones', 'ITEM_SLOT', 'ITEM_SLOT', 'Rhaast', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Kindred', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Alistar', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Zeri', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Morgana', 'Fist of Fairness', 'ITEM_SLOT', 'ITEM_SLOT', 'Varus', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Kobuko', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Predicted token ID at index 87: Aurora
['Braum', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Yuumi', 'Obsidian Cleaver', 'Sunlight Cape', 'Tear of the Goddess', 'Mordekaiser', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Jarvan IV', 'Last Whisper', 'Chalice of Power', 'ITEM_SLOT', 'Shyvana', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Viego', 'Last Whisper', 'ITEM_SLOT', 'ITEM_SLOT', 'Senna', 'Quickestsilver', 'ITEM_SLOT', 'ITEM_SLOT', 'Samira', 'Deathblade', 'ITEM_SLOT', 'ITEM_SLOT', 'Varus', 'ITEM_SLOT',

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    tot_penalty = 0
    for inputs in dataloader:
        input_ids = inputs['input'].to(device)
        expected_tensor = inputs['penalty'].to(device)
        optimizer.zero_grad()
        logits = model(input_ids)
        loss= custom_loss(logits, decision+3, expected_tensor)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

Epoch 1/5 - Loss: 1.8613
Epoch 2/5 - Loss: 1.8604
Epoch 3/5 - Loss: 1.8599
Epoch 4/5 - Loss: 1.8594
Epoch 5/5 - Loss: 1.8589


In [ ]:
for i in range(5):
  board = make_board()
  print(board[1:hero_boundary])
  input_ids = [vocab[t] for t in board]
  id = evaluate(model, torch.tensor(input_ids).unsqueeze(0).to(device), decision+3)

['Rhaast', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Illaoi', 'Tear of the Goddess', 'ITEM_SLOT', 'ITEM_SLOT', 'Shyvana', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Miss Fortune', 'Spectral Cutlass', 'ITEM_SLOT', 'ITEM_SLOT', 'Vex', "Zeke's Herald", 'ITEM_SLOT', 'ITEM_SLOT', 'Renekton', 'Sunlight Cape', 'ITEM_SLOT', 'ITEM_SLOT', 'Ekko', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Rhaast', 'Statikk Shiv', 'ITEM_SLOT', 'ITEM_SLOT', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
Predicted token ID at index 87: Vex
['Alistar', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Aphelios', "Runaan's Hurricane", 'ITEM_SLOT', 'ITEM_SLOT', 'Yuumi', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Vayne', 'Hextech Gunblade', 'ITEM_SLOT', 'ITEM_SLOT', 'Vayne', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Jax', 'ITEM_SLOT', 'ITEM_SLOT', 'ITEM_SLOT', 'Illaoi', 'Steadfast Heart', 'ITEM_SLOT', 'ITEM_SLOT', 'Veigar', 'Giant Slayer', 'ITEM_SLOT', 'ITEM_SLOT', 'Shaco', "The Baron's Gift", 'ITEM_SLOT'

In [ ]:
torch.save(model.state_dict(), "item3_model.pth")